In [1]:
from google.colab import files
uploaded = files.upload()
import pandas as pd # Import the pandas library
df = pd.read_csv('TMSMM_class6_Amazon_Reviews_small.csv')

Saving TMSMM_class6_Amazon_Reviews_small.csv to TMSMM_class6_Amazon_Reviews_small.csv


In [2]:
import pandas as pd
import numpy as np

# load the data
reviews_datasets = pd.read_csv('TMSMM_class6_Amazon_Reviews_small.csv')
reviews_datasets = reviews_datasets.head(450)
reviews_datasets.dropna()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text;;
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...
16,17,B001GVISJM,A3KLWF6WQ5BNYO,Erica Neathery,0,0,2,1348099200,poor taste,I love eating them and they are good for watch...
...,...,...,...,...,...,...,...,...,...,...
414,420,B004X8TK8I,A3GIMXLOEPFJBV,Samuel Spade,0,0,4,1342396800,These are Famous for a reason.,These arrived on time and packaged in green pl...
433,439,B000G6RYNE,A926YP1BSD72Y,Shopper11,0,0,5,1321747200,Best gluten-free dairy-free chips,These are my favorite gluten-free dairy-free f...
437,443,B000G6RYNE,A391CLP4UMXJQG,Dane King,0,0,4,1300924800,Very good.,Salt and vinegar chips are definitely my favor...
442,448,B000G6RYNE,ACFPHOYVB71V4,karob,0,0,5,1292630400,BEST BUY in BBQ Chips,Tried many varieties of BBQ chips and hands do...


In [3]:
# use TFIDF vectorizer since NMF works with TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(reviews_datasets['Text;;'].values.astype('U'))

In [4]:
# we can create a probability matrix that contains probabilities of all the words in the vocabulary for all the topics
from sklearn.decomposition import NMF

nmf = NMF(n_components=5, random_state=42)
nmf.fit(doc_term_matrix )

NMF(n_components=5, random_state=42)

In [8]:
# randomly get 5 words from our vocabulary
import random

for i in range(5):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names_out())) # Use get_feature_names_out() instead of get_feature_names()
    print(tfidf_vect.get_feature_names_out()[random_id]) # Use get_feature_names_out() instead of get_feature_names()

amazon
lot
make
prepare
lot


In [10]:
# retrieve the probability vector of words for the first topic and will retrieve the indexes of the five words with the highest probabilities
first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-5:]

In [12]:
# tfidf_vect object to retrieve the actual words
for i in top_topic_words:
    print(tfidf_vect.get_feature_names_out()[i])


flavor
tea
like
just
taste


In [14]:
# print the five words with highest probabilities for each of the topics
for i, topic in enumerate(nmf.components_):
    print(f'Top five words for topic #{i}:')
    print([tfidf_vect.get_feature_names_out()[i] for i in topic.argsort()[-5:]])
    print('\n')

Top five words for topic #0:
['flavor', 'tea', 'like', 'just', 'taste']


Top five words for topic #1:
['buying', 'candy', 'price', 'store', 'product']


Top five words for topic #2:
['different', 'balance', 'natural', 'dog', 'food']


Top five words for topic #3:
['time', 'grocery', 'like', 'tastes', 'br']


Top five words for topic #4:
['ham', 'likes', 'sugar', 'amazon', 'great']




In [15]:
# adds the topics to the data set and displays the first five rows
topic_values = nmf.transform(doc_term_matrix)
reviews_datasets['Topic'] = topic_values.argmax(axis=1)